웹상에서 자바스크립트를 통해서 정보를 보내고 처리하는 웹을 스크롤링하기

* 개발자도구 < network < headers가 GET 요청일 경우, Request URL 주소가 그대로 보내진다.
* 여기에서는 이 주소를 흉내내어 크롤링을 시도한다.

In [1]:
import requests

page_url = "https://m.search.naver.com/p/csearch/ocontent/util/SpellerProxy"
params = {
    #"_callback": "jQuery11240733170739760759_1571369758105",   # & 구분  JSONP 형식. 생략 가능
    '_callback': "_",
    "q": "아버지가방에들어가신다.", 
    "where": "nexearch", 
    "color_blindness": 0, 
    #"_": 1571369758107  # _ 는 time stamp 생략 가능
}  
res = requests.get(page_url, params=params)
res

<Response [200]>

In [2]:
import time
time.time()

1571381418.803977

In [3]:
import re
json_string = res.text
json_string = re.sub(r'(^_\(|\);$)', '', json_string)
json_string  # callback의 _ 삭제

'{"message":{"result":{"errata_count":1,"origin_html":"<span class=\'result_underline\'>아버지가방에들어가신다.</span>","html":"<em class=\'green_text\'>아버지가 방에 들어가신다.</em>","notag_html":"아버지가 방에 들어가신다."}}}'

### JSONP
* 브라우저의 Same-Origin 정책으로 인해 동일 도메인에 대해서만 Ajax 요청 가능
* 이 때 JSONP는 다른 도메인의 서버로 데이터를 요청하기 위해 사용

In [5]:
import json
import re
import requests
from bs4 import BeautifulSoup

def naver_spelling_check(q):
    url = "https://m.search.naver.com/p/csearch/ocontent/util/SpellerProxy"
    params = {
    #"_callback": "jQuery11240733170739760759_1571369758105",   # & 구분  JSONP 형식. 생략 가능
    '_callback': "_",
    "q": q, 
    "where": "nexearch", 
    "color_blindness": 0, 
    #"_": 1571369758107  # _ 는 time stamp 생략 가능
}  
    res = requests.get(url, params=params)
    jsonp_string = res.text
    jsonp_string = re.sub(r'(^_\(|\);$)', "", jsonp_string)  # 앞부분의 '_(' 와 끝의 ');' 삭제  ^:시작을 의미
    obj = json.loads(jsonp_string)
    return obj['message']['result']

In [6]:
naver_spelling_check("아버지가들어가신다")

{'errata_count': 1,
 'origin_html': "<span class='result_underline'>아버지가들어가신다</span>",
 'html': "<em class='green_text'>아버지가 들어가신다</em>",
 'notag_html': '아버지가 들어가신다'}

In [ ]:
# 정규표현식을 쓰지 않고 간단하게 처리하는 방법
jsonp_string = res.text
jsonp_string = jsonp_string.replace('_(', "").replace(');', "")

In [ ]:
json.loads(jsonp_string)  # json 포맷 가능